# Comparing blood plasma, blood serum, blood vessel endothelium, and monocyte data

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import Classification_Utils as cu
import MaxQuant_Postprocessing_Functions as mq
import numpy as np
from os import listdir
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.externals import joblib
import time

## Load and clean data

In [4]:
BLOOD_DERIVATIVES_DIR = 'F:\Blood_Derivatives\\'

files_dir = BLOOD_DERIVATIVES_DIR
file_paths = listdir(files_dir) 

df = cu.combine_csvs(files_dir, file_paths)

In [ ]:
mq.log2_normalize(df)

df_min = df.min().min()
impute_val = df_min/2
df = df.fillna(impute_val)

# median normalize
mq.median_normalize(df)

## Map each column to a corresponding label

In [ ]:
tissues = ['Blood_Plasma', 'Blood_Serum', 'Blood_Vessel_Endothelium', 'Monocyte']
 
tissues_to_columns = cu.map_tissues_to_columns(df, tissues)

In [ ]:
column_names = df.columns.values.tolist()
labels = cu.get_labels(column_names, tissues_to_columns)

## PCA plot, Pearson

## Classify

## Find distinguishing peptides